In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd 
import tensorflow as tf# data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# Any results you write to the current directory are saved as output.

In [ ]:
from __future__ import print_function
from tensorflow.keras.callbacks import EarlyStopping
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation,MaxPooling2D
from keras.layers import AveragePooling2D, Input, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
import numpy as np
import os
from keras.layers import Input
import sklearn
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

from keras.optimizers import SGD 
from keras.callbacks import LearningRateScheduler
from keras.optimizers import *
from keras.models import Model,Sequential
from keras.layers import *
from keras.activations import *
from keras.callbacks import * 
from numpy import zeros, newaxis

In [ ]:
labels_train=[]
labels_train_abnormal = pd.DataFrame.to_numpy(pd.read_csv("../input/mrnet-v1/MRNet-v1.0/train-abnormal.csv")) 
labels_train_acl = pd.DataFrame.to_numpy(pd.read_csv("../input/mrnet-v1/MRNet-v1.0/train-acl.csv")) 
#labels_train_men = pd.DataFrame.to_numpy(pd.read_csv("../input/mrnet-v1/MRNet-v1.0/train-meniscus.csv")) 
labels_train.append(1)
labels_train.append(0)
#labels_train.append(0)

for i in range (0,1129):
    labels_train.append(labels_train_abnormal[i,1])
    labels_train.append(labels_train_acl[i,1])
    #labels_train.append(labels_train_men[i,1])
labels_train=np.array(labels_train).reshape(-1, 2)
labels_train

# 1. ****#Loading Train Datasets in shape(1130,16,256,256) and label (1130,2)

In [ ]:
X_Train_ax = []
Y_Train_ax=[]
X_Train_cor=[]
Y_Train_cor=[]
X_Train_sag=[]
Y_Train_sag=[]

for patient_ID in range(1130):
    label=labels_train[patient_ID]
    if(patient_ID<10):
        pathd1='../input/mrnet-v1/MRNet-v1.0/train/' + 'axial/000'+str(patient_ID)+'.npy'
        pathd2='../input/mrnet-v1/MRNet-v1.0/train/' + 'coronal/000'+str(patient_ID)+'.npy'
        pathd3='../input/mrnet-v1/MRNet-v1.0/train/' + 'sagittal/000'+str(patient_ID)+'.npy'
    elif(patient_ID<100):
        pathd1='../input/mrnet-v1/MRNet-v1.0/train/' + 'axial/00'+str(patient_ID)+'.npy'
        pathd2='../input/mrnet-v1/MRNet-v1.0/train/' + 'coronal/00'+str(patient_ID)+'.npy'
        pathd3='../input/mrnet-v1/MRNet-v1.0/train/' + 'sagittal/00'+str(patient_ID)+'.npy'
    elif(patient_ID<1000):
        pathd1='../input/mrnet-v1/MRNet-v1.0/train/' + 'axial/0'+str(patient_ID)+'.npy'
        pathd2='../input/mrnet-v1/MRNet-v1.0/train/' + 'coronal/0'+str(patient_ID)+'.npy'
        pathd3='../input/mrnet-v1/MRNet-v1.0/train/' + 'sagittal/0'+str(patient_ID)+'.npy'
    else:
        pathd1='../input/mrnet-v1/MRNet-v1.0/train/' + 'axial/'+str(patient_ID)+'.npy'
        pathd2='../input/mrnet-v1/MRNet-v1.0/train/' + 'coronal/'+str(patient_ID)+'.npy'
        pathd3='../input/mrnet-v1/MRNet-v1.0/train/' + 'sagittal/'+str(patient_ID)+'.npy'

    d1Image = np.load(pathd1) # this part for reading 3 prespectives of each patient with multi-layers each
    d2Image = np.load(pathd2)
    d3Image = np.load(pathd3)
    startd1=int((d1Image.shape[0]/2)-8)  #this part to capture layers of interest 
    endd1=int((d1Image.shape[0]/2)+8)
    startd2=int((d2Image.shape[0]/2)-8)
    endd2=int((d2Image.shape[0]/2)+8)
    startd3=int((d3Image.shape[0]/2)-8)
    endd3=int((d3Image.shape[0]/2)+8)

    
    image_tensor=d1Image[startd1:endd1,:,:]
    #.reshape(256,256,16)
    X_Train_ax.append(image_tensor)
    Y_Train_ax.append(label)
    image_tensor2=d2Image[startd2:endd2,:,:]
    #.reshape(256,256,16)
    X_Train_cor.append(image_tensor2)
    Y_Train_cor.append(label)
    image_tensor3=d3Image[startd3:endd3,:,:]
    #.reshape(256,256,16)
    X_Train_sag.append(image_tensor3)
    Y_Train_sag.append(label)

    
print(np.asarray(X_Train_ax).shape)
print(np.asarray(Y_Train_ax).shape)
print(np.asarray(X_Train_cor).shape)
print(np.asarray(Y_Train_cor).shape)
print(np.asarray(X_Train_sag).shape)
print(np.asarray(Y_Train_sag).shape)
X_Train_ax = np.array(X_Train_ax)
Y_Train_ax = np.array(Y_Train_ax)
X_Train_cor = np.array(X_Train_cor)
Y_Train_cor = np.array(Y_Train_cor)
X_Train_sag = np.array(X_Train_sag)
Y_Train_sag = np.array(Y_Train_sag)

# Loading test dataset

In [ ]:
labels_test=[]
labels_test_abnormal = pd.DataFrame.to_numpy(pd.read_csv("../input/mrnet-v1/MRNet-v1.0/valid-abnormal.csv")) 
labels_test_acl = pd.DataFrame.to_numpy(pd.read_csv("../input/mrnet-v1/MRNet-v1.0/valid-acl.csv")) 
#labels_test_men = pd.DataFrame.to_numpy(pd.read_csv("../input/mrnet-v1/MRNet-v1.0/valid-meniscus.csv")) 
labels_test.append(0)
labels_test.append(0)
#labels_test.append(0)

for i in range (119):
    labels_test.append(labels_test_abnormal[i,1])
    labels_test.append(labels_test_acl[i,1])
    #labels_test.append(labels_test_men[i,1])
labels_test=np.array(labels_test).reshape(-1, 2)
labels_test

#from PIL import Image
X_Test_ax = []
Y_Test_ax=[]
X_Test_cor=[]
Y_Test_cor=[]
X_Test_sag=[]
Y_Test_sag=[]

for patient_ID in range(119):
    label=labels_test[patient_ID]
    patient_ID=1130+patient_ID
    if(patient_ID<10):
        pathd1='../input/mrnet-v1/MRNet-v1.0/valid/' + 'axial/000'+str(patient_ID)+'.npy'
        pathd2='../input/mrnet-v1/MRNet-v1.0/valid/' + 'coronal/000'+str(patient_ID)+'.npy'
        pathd3='../input/mrnet-v1/MRNet-v1.0/valid/' + 'sagittal/000'+str(patient_ID)+'.npy'
    elif(patient_ID<100):
        pathd1='../input/mrnet-v1/MRNet-v1.0/valid/' + 'axial/00'+str(patient_ID)+'.npy'
        pathd2='../input/mrnet-v1/MRNet-v1.0/valid/' + 'coronal/00'+str(patient_ID)+'.npy'
        pathd3='../input/mrnet-v1/MRNet-v1.0/valid/' + 'sagittal/00'+str(patient_ID)+'.npy'
    elif(patient_ID<1000):
        pathd1='../input/mrnet-v1/MRNet-v1.0/valid/' + 'axial/0'+str(patient_ID)+'.npy'
        pathd2='../input/mrnet-v1/MRNet-v1.0/valid/' + 'coronal/0'+str(patient_ID)+'.npy'
        pathd3='../input/mrnet-v1/MRNet-v1.0/valid/' + 'sagittal/0'+str(patient_ID)+'.npy'
    else:
        pathd1='../input/mrnet-v1/MRNet-v1.0/valid/' + 'axial/'+str(patient_ID)+'.npy'
        pathd2='../input/mrnet-v1/MRNet-v1.0/valid/' + 'coronal/'+str(patient_ID)+'.npy'
        pathd3='../input/mrnet-v1/MRNet-v1.0/valid/' + 'sagittal/'+str(patient_ID)+'.npy'

    d1Image = np.load(pathd1)
    d2Image = np.load(pathd2)
    d3Image = np.load(pathd3)
    startd1=int((d1Image.shape[0]/2)-8)
    endd1=int((d1Image.shape[0]/2)+8)
    startd2=int((d2Image.shape[0]/2)-8)
    endd2=int((d2Image.shape[0]/2)+8)
    startd3=int((d3Image.shape[0]/2)-8)
    endd3=int((d3Image.shape[0]/2)+8)

    
    image_tensor=d1Image[startd1:endd1,:,:]
    #.reshape(256,256,16)
    X_Test_ax.append(image_tensor)
    Y_Test_ax.append(label)
    image_tensor2=d2Image[startd2:endd2,:,:]
    #.reshape(256,256,16)
    X_Test_cor.append(image_tensor2)
    Y_Test_cor.append(label)
    image_tensor3=d3Image[startd3:endd3,:,:]
    #.reshape(256,256,16)
    X_Test_sag.append(image_tensor3)
    Y_Test_sag.append(label)

    
    
    
print(np.asarray(X_Test_ax).shape)
print(np.asarray(Y_Test_ax).shape)
print(np.asarray(X_Test_cor).shape)
print(np.asarray(Y_Test_cor).shape)
print(np.asarray(X_Test_sag).shape)
print(np.asarray(Y_Test_sag).shape)
X_Test_ax = np.array(X_Test_ax)
Y_Test_ax = np.array(Y_Test_ax)
X_Test_cor = np.array(X_Test_cor)
Y_Test_cor = np.array(Y_Test_cor)
X_Test_sag = np.array(X_Test_sag)
Y_Test_sag = np.array(Y_Test_sag)

In [ ]:
labels_train.shape
#X_Train_ax.shape

In [ ]:
def mri_predict(axial,coronal,sagittal):
    axial = axial.reshape(16,256,256,1)
    coronal = coronal.reshape(16,256,256,1)
    sagittal = sagittal.reshape(16,256,256,1)
    
    axial_cnn = keras.models.load_model("../input/ax-cnn/axcnn.h5")
    coronal_cnn = keras.models.load_model("../input/ax-cnn/corpart.h5")
    sagittal_cnn = keras.models.load_model("../input/ax-cnn/sagpart.h5")
    
    ax_prediction = 0
    cor_prediction = 0
    sag_prediction = 0
    
    for i in range(16):
        axialtemp = axial[i,:,:,:]
        axialtemp = axialtemp.reshape(1,256,256,1)
        ax_predict = axial_cnn.predict(axialtemp)
        ax_prediction+=ax_predict 
        
        coronaltemp = coronal[i,:,:,:]
        coronaltemp = coronaltemp.reshape(1,256,256,1)
        cor_predict = coronal_cnn.predict(coronaltemp)
        cor_prediction+=cor_predict
        
        sagittaltemp = sagittal[i,:,:,:]
        sagittaltemp = sagittaltemp.reshape(1,256,256,1)
        sag_predict = sagittal_cnn.predict(sagittaltemp)
        sag_prediction+=sag_predict
        
    fp_sag = sag_prediction/16
    fp_ax = ax_prediction/16
    fp_cor = cor_prediction/16
    return(fp_sag,fp_ax,fp_cor)

In [ ]:
fp_sag,fp_ax,fp_cor = mri_predict(X_Train_ax[0],X_Train_cor[0],X_Train_sag[0])

In [ ]:
print(fp_sag,fp_ax,fp_cor)

In [ ]:
fp_ax = np.array(fp_ax)
fp_cor = np.array(fp_cor)
fp_sag = np.array(fp_sag)

In [ ]:
combined = np.concatenate((fp_ax,fp_cor,fp_sag),axis = 1)

In [ ]:
combined

# Converting to single array of multiple slices

1. xtax: Axial train
2. ytax: Axial train label
3. xtsag: Sagittal train
4. ytsag: Sagittal train label
5. xtcor: Coronal train
6. ytcor: Coronal train label


In [ ]:
xtax = []
ytax = []
for i in range(1130):
    for j in range(16):
        xtax.append(X_Train_ax[i,j,:,:])
        ytax.append(Y_Train_ax[i,:])
        
xtax = np.array(xtax)        
ytax = np.array(ytax)

In [ ]:
print("Shape of the axial plane training dataset = "+ str(xtax.shape))
print("Shape of the axial plane training dataset label= "+ str(ytax.shape))

In [ ]:
xtax = xtax.reshape(18080,256,256,1)
ytax = ytax.reshape(18080,2,1)

# **Preparing Test Dataset**

1. xtestax: Axial test
2. ytestax: Axial test label
3. xtestsag: Sagittal test
4. ytestsag: Sagittal test label
5. xtestcor: Coronal test
6. ytestcor: Coronal test label


In [ ]:
xtestax = []
ytestax = []

for i in range(119):
    for j in range(16):
        xtestax.append(X_Test_ax[i,j,:,:])
        ytestax.append(Y_Test_ax[i,:])
        
xtestax = np.array(xtestax)        
ytestax = np.array(ytestax)        
print(xtestax.shape,ytestax.shape)

In [ ]:
xtestax = xtestax.reshape(1904,256,256,1)
ytestax = ytestax.reshape(1904,2,1)

# A slice

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(xtax[1600,:,:,0],cmap = 'gray')

In [ ]:
np.amax(xtax[10])
xtax = xtax.reshape(18080,256,256,1)
xtestax = xtestax.reshape(1904,256,256,1)

In [ ]:
ytax.shape

**Converting the axial label from 2 value label to 3 value label**

**Label format**: [Normal,Abnormal,ACL]

In [ ]:
ytaxf=[]
for i in range(18080):
    if(ytax[i,0,0]==1):
        temp = 0,ytax[i,0,0],ytax[i,1,0]
        ytaxf.append(temp)
    else:
        temp = 1,ytax[i,0,0],ytax[i,1,0]
        ytaxf.append(temp)
ytaxf = np.array(ytaxf)
print(ytaxf.shape)

In [ ]:
ytestaxf=[]
for i in range(1904):
    if(ytestax[i,0,0]==1):
        temp = 0,ytestax[i,0,0],ytestax[i,1,0]
        ytestaxf.append(temp)
    else:
        temp = 1,ytestax[i,0,0],ytestax[i,1,0]
        ytestaxf.append(temp)
ytestaxf = np.array(ytestaxf)
print(ytestaxf.shape)

# Seperating the ACL slices from axial training set for augmentation

In [ ]:
indexofacl=[]
for i in range(18080):
    if ytaxf[i,0]==0 and ytaxf[i,1]==1 and ytaxf[i,2]==1:
        indexofacl.append(i)
len(indexofacl)

In [ ]:
count = 0
for i in indexofacl:
    if i<=4000:
        count+=1
print(count)

In [ ]:
#acl seperate
acldata = []
acldatalabel = []
for i in indexofacl:
    acldata.append(xtax[i,:,:,0])
    acldatalabel.append(ytaxf[i,:])

In [ ]:
acldata = np.array(acldata)
acldatalabel = np.array(acldatalabel)
print(acldata.shape,acldatalabel.shape)

In [ ]:
np.array_equal(acldata[5],acldata[6])

In [ ]:
acldatar = acldata.reshape(3328,256,256,1)

In [ ]:
indexofnormal=[]
for i in range(18080):
    if ytaxf[i,0]==1 and ytaxf[i,1]==0 and ytaxf[i,2]==0:
        indexofnormal.append(i)
len(indexofnormal)

In [ ]:
normaldata = []
normaldatalabel = []
for i in indexofnormal:
    normaldata.append(xtax[i,:,:,0])
    normaldatalabel.append(ytaxf[i,:])
normaldata = np.array(normaldata)
normaldatalabel = np.array(normaldatalabel)
print(normaldata.shape,normaldatalabel.shape)    

normaldatar = normaldata.reshape(3472,256,256,1)

In [ ]:
normaldatar.shape

In [ ]:
'''acldatar[0].shape

# **ACL slices Augmentation**

In [ ]:
#Data Augmentatiion
datagen = ImageDataGenerator(rotation_range = 25, fill_mode = 'nearest')
datagen.fit(acldatar)
x_batch_axial= acldatar
y_batch_axial = acldatalabel


In [ ]:
epochs = 1
for e in range(epochs):
    print('Epoch', e)
    batches = 0
    per_batch = 5
    for x_batch, y_batch in datagen.flow(acldatar, acldatalabel, batch_size=per_batch):
        x_batches = np.concatenate((x_batch_axial, x_batch), axis = 0)
        y_batches = np.concatenate((y_batch_axial, y_batch), axis = 0)
        batches += 1
        if batches >= len(acldatar) / per_batch:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break

In [ ]:
#shape of the augmented data array
x_batches.shape

In [ ]:
np.array_equal(x_batches[20],x_batches[21]) #this is to check if 2 slices are same(augmented data)

In [ ]:

'''print(xtax.shape,x_batches.shape)

Sample augmented data

In [ ]:
'''import matplotlib.pyplot as plt
plt.imshow(x_batches[206,:,:,0],cmap = 'gray')

In [ ]:
'''np.save('axialdata',arr = xtax)
np.save('augmented_data',arr = x_batches) 

In [ ]:
'''from IPython.display import FileLink
FileLink(r'axialdata.npy')

In [ ]:
#FileLink(r'augmented_data.npy')
import gc
gc.collect()

In [ ]:
#del x_batch_axial

Combine the augmented dataset with the existing dataset and divide to 2 parts(direct concatenation is not possible since very huge arrays)

In [ ]:
#combining part 1
axial_train_1 = np.concatenate((xtax[0:3500],acldatar[0:2500],normaldatar[:1000]))

In [ ]:
#combining part 2
axial_train_2 = np.concatenate((xtax[4000:5000],acldatar[2000:]))

In [ ]:
#combining part 3
axial_train_3 = xtax[8000:12000]

In [ ]:
#combining part 4
axial_train_4 = xtax[12000:]

In [ ]:
#combining part 1 label
axial_train_label_1 = np.concatenate((ytaxf[0:3500],acldatalabel[0:2500],normaldatalabel[:1000]))

In [ ]:
#combining part 2 label
axial_train_label_2 = np.concatenate((ytaxf[4000:5000],acldatalabel[2000:]))

In [ ]:
#combining part 3 label
axial_train_label_3 = ytaxf[8000:12000]

In [ ]:
#combining part 4 label
axial_train_label_4 = ytaxf[12000:]

In [ ]:
print(axial_train_1.shape,axial_train_label_1.shape)

In [ ]:
#randomizing part 1
axial_train_1 , axial_train_label_1 = shuffle(axial_train_1 , axial_train_label_1)

In [ ]:
#randomizing part 2
axial_train_2 , axial_train_label_2 = shuffle(axial_train_2 , axial_train_label_2)

In [ ]:
#randomizing part 3
axial_train_3 , axial_train_label_3 = shuffle(axial_train_3 , axial_train_label_3)

In [ ]:
#randomizing part 4
axial_train_4 , axial_train_label_4 = shuffle(axial_train_4 , axial_train_label_4)

In [ ]:
xtesttax = np.concatenate((xtax[:1654],xtestax[:250]))

In [ ]:
ytesttaxf = np.concatenate((ytaxf[:1654],ytestaxf[:20]))

In [ ]:
'''#combining part 2
axial_train_2 = np.concatenate((xtax[8580:],x_batches[2000:]))
axial_train_label_2 = np.concatenate((ytaxf[8580:],y_batches[2000:]))


In [ ]:
'''#randomizing part 2
axial_train_2 , axial_train_label_2 = shuffle(axial_train_2 , axial_train_label_2)

print("Shape of the axial plane training dataset = "+ str(xtax.shape))
print("Shape of the axial plane training dataset label= "+ str(ytaxf.shape))
print("Shape of the axial plane test dataset = "+ str(xtestax.shape))
print("Shape of the axial plane test dataset label= "+ str(ytestaxf.shape))

ytestaxf[1401]

In [ ]:
def ax_cnn(input_shape):
    X_input = Input(input_shape)
    X = Conv2D(28,(5,5),strides = (1,1),name = 'conv0')(X_input)
    X = BatchNormalization(axis = 3, name ='bn0')(X)
    X = Activation('relu')(X)
    X = Dropout(0.2)(X)
    
    X = AveragePooling2D((7,7), strides= (2,2), name = 'avgpool1')(X)
    X = Activation('relu')(X)
    X = Dropout(0.2)(X)
    
    X = Conv2D(10,(5,5),strides = (1,1),name = 'conv1')(X)
    X = BatchNormalization(axis = 3, name ='bn1')(X)
    X = Activation('relu')(X)
    X = Dropout(0.2)(X)
    X = AveragePooling2D((5,5), strides= (2,2), name = 'avgpool2')(X)
    X = Activation('relu')(X)
    X = Dropout(0.2)(X)
    
    X = Conv2D(1,(5,5),strides = (1,1),name = 'conv2')(X)
    X = BatchNormalization(axis = 3, name ='bn2')(X)
    X = Activation('relu')(X)
    X = Dropout(0.2)(X)
    
    X = Flatten()(X)
    X = Dense(256,activation = 'sigmoid',name = 'd1')(X)
    X = Dense(64,activation = 'sigmoid',name = 'd2')(X)
    X = Dense(3,activation = 'sigmoid',name = 'd3')(X)
    model = Model(inputs = X_input, outputs = X, name='ax_cnn')
    return model

In [ ]:
sagcnn.summary()

In [ ]:
axcnn = ax_cnn(xtax.shape[1:])

In [ ]:
sagcnn = ax_cnn([256,256,1])

axcnn = ax_cnn(xtax.shape[1:])

In [ ]:
keras.optimizers.Adam(
   learning_rate = 0.01, beta_1 = 0.9, beta_2 = 0.999, amsgrad = False
)

In [ ]:
axcnn.compile('adam', 'mean_squared_error',metrics = ['accuracy'])

axcnn.compile('adam', 'binary_crossentropy',metrics = ['accuracy'])

In [ ]:
axcnn.fit(axial_train_1,axial_train_label_1,epochs = 1, batch_size = 100)

In [ ]:
axcnn2.fit(axial_train_2,axial_train_label_2,epochs = 5, batch_size = 50)

In [ ]:
axcnn1.fit(axial_train_3,axial_train_label_3,epochs = 5, batch_size = 50)

In [ ]:
axcnn1.fit(axial_train_4,axial_train_label_4,epochs = 7, batch_size = 50)

In [ ]:
axcnn.save("/kaggle/working/axcnn_new.h5")

In [ ]:
axcnn = keras.models.load_model("../input/ax-cnn/axcnn.h5")

In [ ]:
from IPython.display import FileLink
FileLink(r'axcnn.h5')

In [ ]:
axcnn1 = keras.models.load_model("/kaggle/working/axcnn1.h5")

axcnn.fit(xtax,ytaxf,epochs = 1, batch_size = 50)

In [ ]:
preds = axcnn.evaluate(xtestax,ytestaxf,batch_size = 50,verbose=1, sample_weight=None)

print ("Loss = " + str(preds[0])) 
print ("Test Accuracy = " + str(preds[1]))

preds = axcnn.evaluate(xtestax,ytestaxf,batch_size = 32,verbose=1, sample_weight=None)

print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
predictimage = xtestax[105]
predictimage=predictimage.reshape(1,256,256,1)
print(axcnn.predict(predictimage))
print(ytestaxf[105])

predictimage = xtestax[890]
predictimage = predictimage.reshape(1,256,256,1)
print(axcnn.predict(predictimage))
print(ytestaxf[800])

ytestaxf[1800]


> ****axcnn.summary()

In [ ]:
axcnn.save("axialmodelpart1.h5")

# ***CORONAL PLANE***

In [ ]:
xtcor = []
ytcor = []
for i in range(1130):
    for j in range(16):
        xtcor.append(X_Train_cor[i,j,:,:])
        ytcor.append(Y_Train_cor[i,:])
        
xtcor = np.array(xtcor)        
ytcor = np.array(ytcor)

In [ ]:
print(xtcor.shape)
print(ytcor.shape)

In [ ]:
xtcor = xtcor.reshape(18080,256,256,1)
ytcor = ytcor.reshape(18080,2,1)

In [ ]:
xtestcor = []
ytestcor = []

for i in range(119):
    for j in range(16):
        xtestcor.append(X_Test_cor[i,j,:,:])
        ytestcor.append(Y_Test_cor[i,:])
        
xtestcor = np.array(xtestcor)        
ytestcor = np.array(ytestcor)        
print(xtestcor.shape,ytestcor.shape)

In [ ]:
xtestcor = xtestcor.reshape(1904,256,256,1)
ytestcor = ytestcor.reshape(1904,2,1)

In [ ]:
np.amax(xtcor[10])
xtcor = xtcor.reshape(18080,256,256,1)
xtestcor = xtestcor.reshape(1904,256,256,1)

In [ ]:
ytcorf=[]
for i in range(18080):
    if(ytcor[i,0,0]==1):
        temp = 0,ytcor[i,0,0],ytcor[i,1,0]
        ytcorf.append(temp)
    else:
        temp = 1,ytcor[i,0,0],ytcor[i,1,0]
        ytcorf.append(temp)
ytcorf = np.array(ytcorf)
print(ytcorf.shape)

In [ ]:
ytestcorf=[]
for i in range(1904):
    if(ytestcor[i,0,0]==1):
        temp = 0,ytestcor[i,0,0],ytestcor[i,1,0]
        ytestcorf.append(temp)
    else:
        temp = 1,ytestcor[i,0,0],ytestcor[i,1,0]
        ytestcorf.append(temp)
ytestcorf = np.array(ytestcorf)
print(ytestcorf.shape)

In [ ]:
ytcorf = ytcorf.reshape(18080,3,1)
ytestcorf = ytestcorf.reshape(1904,3)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(xtcor[16,:,:,0],cmap = 'gray')

In [ ]:
indexofacl=[]
for i in range(18080):
    if ytcorf[i,0]==0 and ytcorf[i,1]==1 and ytcorf[i,2]==1:
        indexofacl.append(i)
len(indexofacl)

In [ ]:
indexofnormal=[]
for i in range(18080):
    if ytcorf[i,0]==1 and ytcorf[i,1]==0 and ytcorf[i,2]==0:
        indexofnormal.append(i)
len(indexofnormal)

In [ ]:
normaldata = []
normaldatalabel = []
for i in indexofnormal:
    normaldata.append(xtcor[i,:,:,0])
    normaldatalabel.append(ytcorf[i,:])

In [ ]:
normaldata = np.array(normaldata)
normaldatalabel = np.array(normaldatalabel)
print(normaldata.shape,normaldatalabel.shape)

In [ ]:
normaldatar = normaldata.reshape(3472,256,256,1)
normaldatalabel = normaldatalabel.reshape(3472,3,1)

In [ ]:
acldata = []
acldatalabel = []
for i in indexofacl:
    acldata.append(xtcor[i,:,:,0])
    acldatalabel.append(ytcorf[i,:])

In [ ]:
acldata = np.array(acldata)
acldatalabel = np.array(acldatalabel)
print(acldata.shape,acldatalabel.shape)

In [ ]:
np.array_equal(acldata[5],acldata[6])

In [ ]:
acldatar = acldata.reshape(3328,256,256,1)
acldatalabel = acldatalabel.reshape(3328,3,1)

In [ ]:
acllabel = []
for i in range(3328):
    acllabel.append([0,0,1])

In [ ]:
acllabel = np.array(acllabel)

In [ ]:
acllabel

In [ ]:
ytcorf

In [ ]:
acllabel = acllabel.reshape(3328,3,1)

In [ ]:
datagen = ImageDataGenerator(rotation_range = 25, fill_mode = 'nearest')
datagen.fit(acldatar)
x_batch_coronal= acldatar
y_batch_coronal = acldatalabel

In [ ]:
epochs = 1
for e in range(epochs):
    print('Epoch', e)
    batches = 0
    per_batch = 5
    for x_batch, y_batch in datagen.flow(acldatar, acldatalabel, batch_size=per_batch):
        x_batches = np.concatenate((x_batch_coronal, x_batch), axis = 0)
        y_batches = np.concatenate((y_batch_coronal, y_batch), axis = 0)
        batches += 1
        if batches >= len(acldatar) / per_batch:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break

In [ ]:
corcnn = keras.models.load_model("../input/ax-cnn/axcnn.h5")

In [ ]:
corcnn.summary()

In [ ]:
x_batches.shape

In [ ]:
np.array_equal(x_batches[20],x_batches[21])

In [ ]:
import gc
gc.collect()

Combining part 1

In [ ]:
coronal_train_1 = np.concatenate((xtcor[0:3000],acldatar[0:2000],normaldatar[0:700]))

In [ ]:
coronal_train_2 = np.concatenate((x_batches[2000:],normaldatar[500:1500]))

In [ ]:
coronal_train_3 = np.concatenate((xtcor[5000:6500],acldatar[2000:]))

In [ ]:
coronal_train_3 = xtcor[6500:7500]

In [ ]:
normaldatalabel = normaldatalabel.reshape(3472,3)

In [ ]:
coronal_train_label_1 = np.concatenate((ytcorf[0:3000],acldatalabel[0:2000],normaldatalabel[0:700]))

In [ ]:
coronal_train_label_1 = coronal_train_label_1.reshape(5700,3)

In [ ]:
coronal_train_label_2 = np.concatenate((y_batches[2000:],normaldatalabel[500:1500]))

In [ ]:
coronal_train_label_3 = ytcorf[6500:7500]

In [ ]:
coronal_train_label_3 = np.concatenate((ytcorf[5000:6500],acllabel[2000:]))

In [ ]:
print(coronal_train_3.shape,coronal_train_label_3.shape)

In [ ]:
coronal_train_1 , coronal_train_label_1 = shuffle(coronal_train_1 , coronal_train_label_1)

In [ ]:
coronal_train_2 , coronal_train_label_2 = shuffle(coronal_train_2 , coronal_train_label_2)

In [ ]:
coronal_train_3 , coronal_train_label_3 = shuffle(coronal_train_3 , coronal_train_label_3)

In [ ]:
xtesttcor = np.concatenate((xtcor[:1500],xtestcor[:404]))

In [ ]:
ytesttcorf = np.concatenate((ytcorf[:1500],ytestcorf[:404]))

In [ ]:
def cor_cnn(input_shape):
    X_input = Input(input_shape)
    X = Conv2D(28,(5,5),strides = (1,1),name = 'conv0')(X_input)
    X = BatchNormalization(axis = 3, name ='bn0')(X)
    X = Activation('relu')(X)
    X = Dropout(0.2)(X)
    
    X = AveragePooling2D((7,7), strides= (2,2), name = 'avgpool1')(X)
    X = Activation('relu')(X)
    X = Dropout(0.2)(X)
    
    X = Conv2D(10,(5,5),strides = (1,1),name = 'conv1')(X)
    X = BatchNormalization(axis = 3, name ='bn1')(X)
    X = Activation('relu')(X)
    X = Dropout(0.2)(X)
    
    X = AveragePooling2D((5,5), strides= (2,2), name = 'avgpool2')(X)
    X = Activation('relu')(X)
    X = Dropout(0.2)(X)
    
    X = Conv2D(1,(5,5),strides = (1,1),name = 'conv2')(X)
    X = BatchNormalization(axis = 3, name ='bn2')(X)
    X = Activation('relu')(X)
    X = Dropout(0.2)(X)
    
    X = Flatten()(X)
    X = Dense(256,activation = 'sigmoid',name = 'd1')(X)
    X = Dense(64,activation = 'sigmoid',name = 'd2')(X)
    X = Dense(3,activation = 'sigmoid',name = 'd3')(X)
    model = Model(inputs = X_input, outputs = X, name='cor_cnn')
    return model

In [ ]:
cnn = Sequential()
cnn.add(Conv2D(32, (3, 3), activation="relu", input_shape=(img_width, img_height, 1)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Conv2D(32, (3, 3), activation="relu", input_shape=(img_width, img_height, 1)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Conv2D(32, (3, 3), activation="relu", input_shape=(img_width, img_height, 1)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Conv2D(64, (3, 3), activation="relu", input_shape=(img_width, img_height, 1)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Conv2D(64, (3, 3), activation="relu", input_shape=(img_width, img_height, 1)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Flatten())
cnn.add(Dense(activation = 'relu', units = 256))
cnn.add(Dense(activation = 'relu', units = 64))
cnn.add(Dense(activation = 'sigmoid', units = 1))

In [ ]:
corcnn = cor_cnn(xtcor.shape[1:])

In [ ]:
corcnn.summary()

In [ ]:
keras.optimizers.Adam(
   learning_rate = 0.005, beta_1 = 0.9, beta_2 = 0.999, amsgrad = False
)

In [ ]:
corcnn.compile('adam', 'mean_squared_error',metrics = ['accuracy'])

In [ ]:
corcnn.fit(coronal_train_1,coronal_train_label_1,epochs = 3, batch_size = 50)

In [ ]:
preds = corcnn.evaluate(xtestcor,ytestcorf,batch_size = 50,verbose=1, sample_weight=None)

print ("Loss = " + str(preds[0])) 
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
predictimage = xtestcor[100]
predictimage=predictimage.reshape(1,256,256,1)
print(corcnn.predict(predictimage))
print(ytestcorf[100])

In [ ]:
corcnn.save("/kaggle/working/corcnn_new.h5")

In [ ]:
corcnn = keras.models.load_model("../input/ax-cnn/corcnn_new.h5")

In [ ]:
corcnn.summary()

In [ ]:
from IPython.display import FileLink
FileLink(r'corpart.h5')

# ***Sagittal Plane***

In [ ]:
xtsag = []
ytsag = []
for i in range(1130):
    for j in range(16):
        xtsag.append(X_Train_sag[i,j,:,:])
        ytsag.append(Y_Train_sag[i,:])
        
xtsag = np.array(xtsag)        
ytsag = np.array(ytsag)

In [ ]:
xtsag = xtsag.reshape(18080,256,256,1)
ytsag = ytsag.reshape(18080,2,1)

In [ ]:
xtestsag = []
ytestsag = []

for i in range(119):
    for j in range(16):
        xtestsag.append(X_Test_sag[i,j,:,:])
        ytestsag.append(Y_Test_sag[i,:])
        
xtestsag = np.array(xtestsag)        
ytestsag = np.array(ytestsag)        
print(xtestsag.shape,ytestsag.shape)

In [ ]:
xtestsag = xtestsag.reshape(1904,256,256,1)
ytestsag = ytestsag.reshape(1904,2,1)

In [ ]:
ytsagf=[]
for i in range(18080):
    if(ytsag[i,0,0]==1):
        temp = 0,ytsag[i,0,0],ytsag[i,1,0]
        ytsagf.append(temp)
    else:
        temp = 1,ytsag[i,0,0],ytsag[i,1,0]
        ytsagf.append(temp)
ytsagf = np.array(ytsagf)
print(ytsagf.shape)

In [ ]:
ytestsagf=[]
for i in range(1904):
    if(ytestsag[i,0,0]==1):
        temp = 0,ytestsag[i,0,0],ytestsag[i,1,0]
        ytestsagf.append(temp)
    else:
        temp = 1,ytestsag[i,0,0],ytestsag[i,1,0]
        ytestsagf.append(temp)
ytestsagf = np.array(ytestsagf)
print(ytestsagf.shape)

In [ ]:
indexofacl=[]
for i in range(18080):
    if ytsagf[i,0]==0 and ytsagf[i,1]==1 and ytsagf[i,2]==1:
        indexofacl.append(i)
len(indexofacl)

In [ ]:
acldata = []
acldatalabel = []
for i in indexofacl:
    acldata.append(xtsag[i,:,:,0])
    acldatalabel.append(ytsagf[i,:])

In [ ]:
acldata = np.array(acldata)
acldatalabel = np.array(acldatalabel)
print(acldata.shape,acldatalabel.shape)

In [ ]:
acldatar = acldata.reshape(3328,256,256,1)

In [ ]:
indexofnormal=[]
for i in range(18080):
    if ytsagf[i,0]==1 and ytsagf[i,1]==0 and ytsagf[i,2]==0:
        indexofnormal.append(i)
len(indexofnormal)

In [ ]:
normaldata = []
normaldatalabel = []
for i in indexofnormal:
    normaldata.append(xtsag[i,:,:,0])
    normaldatalabel.append(ytsagf[i,:])

In [ ]:
normaldata = np.array(normaldata)
normaldatalabel = np.array(normaldatalabel)
print(normaldata.shape,normaldatalabel.shape)

In [ ]:
normaldatar = normaldata.reshape(3472,256,256,1)

In [ ]:
datagen = ImageDataGenerator(rotation_range = 25, fill_mode = 'nearest')
datagen.fit(acldatar)
x_batch_coronal= acldatar
y_batch_coronal = acldatalabel

In [ ]:
epochs = 1
for e in range(epochs):
    print('Epoch', e)
    batches = 0
    per_batch = 5
    for x_batch, y_batch in datagen.flow(acldatar, acldatalabel, batch_size=per_batch):
        x_batches = np.concatenate((x_batch_coronal, x_batch), axis = 0)
        y_batches = np.concatenate((y_batch_coronal, y_batch), axis = 0)
        batches += 1
        if batches >= len(acldatar) / per_batch:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break

In [ ]:
sagittal_train_1 = np.concatenate((xtsag[0:3000],acldatar[0:2500],normaldatar[0:1000]))

In [ ]:
sagittal_train_2 = np.concatenate((xtsag[2000:4000],acldatar[2000:],normaldatar[500:1000]))

In [ ]:
sagittal_train_3 = np.concatenate((xtsag[4000:6000],normaldatar[1000:1500]))

In [ ]:
sagittal_train_label_1 = np.concatenate((ytsagf[0:3000],acldatalabel[0:2500],normaldatalabel[0:1000]))

In [ ]:
sagittal_train_label_2 = np.concatenate((ytsagf[2000:4000],acldatalabel[2000:],normaldatalabel[500:1000]))

In [ ]:
sagittal_train_label_3 = np.concatenate((ytsagf[4000:6000],normaldatalabel[1000:1500]))

In [ ]:
sagittal_train_1 , sagittal_train_label_1 = shuffle(sagittal_train_1 , sagittal_train_label_1)

In [ ]:
sagittal_train_2 , sagittal_train_label_2 = shuffle(sagittal_train_2 , sagittal_train_label_2)

In [ ]:
sagittal_train_3 , sagittal_train_label_3 = shuffle(sagittal_train_3 , sagittal_train_label_3)

In [ ]:
sagittal_train_3 , sagittal_train_label_3 = shuffle(sagittal_train_3 , sagittal_train_label_3)

In [ ]:
xtesttsag = np.concatenate((xtsag[:1600],xtestsag[:304]))

In [ ]:
ytesttsagf = np.concatenate((ytsagf[:1600],ytestsagf[:304]))

In [ ]:
def sag_cnn(input_shape):
    X_input = Input(input_shape)
    X = Conv2D(28,(5,5),strides = (1,1),name = 'conv0')(X_input)
    X = BatchNormalization(axis = 3, name ='bn0')(X)
    X = Activation('relu')(X)
    X = Dropout(0.4)(X)
    
    X = AveragePooling2D((7,7), strides= (2,2), name = 'avgpool1')(X)
    X = Activation('relu')(X)
    X = Dropout(0.4)(X)
    
    X = Conv2D(10,(5,5),strides = (1,1),name = 'conv1')(X)
    X = BatchNormalization(axis = 3, name ='bn1')(X)
    X = Activation('relu')(X)
    X = Dropout(0.4)(X)
    
    X = AveragePooling2D((5,5), strides= (2,2), name = 'avgpool2')(X)
    X = Activation('relu')(X)
    X = Dropout(0.4)(X)
    
    X = Conv2D(1,(5,5),strides = (1,1),name = 'conv2')(X)
    X = BatchNormalization(axis = 3, name ='bn2')(X)
    X = Activation('relu')(X)
    X = Dropout(0.4)(X)
    
    X = Flatten()(X)
    X = Dense(256,activation = 'sigmoid',name = 'd1')(X)
    X = Dense(64,activation = 'sigmoid',name = 'd2')(X)
    X = Dense(3,activation = 'sigmoid',name = 'd3')(X)
    model = Model(inputs = X_input, outputs = X, name='sag_cnn')
    return model

In [ ]:
sagcnn = sag_cnn([256,256,1])

In [ ]:
sagcnn.summary()

In [ ]:
sagcnn.compile('adam', 'mean_squared_error',metrics = ['accuracy'])

In [ ]:
sagcnn.fit(sagittal_train_1,sagittal_train_label_1,epochs = 3, batch_size = 50)

In [ ]:
preds = sagcnn.evaluate(xtestsag,ytestsagf,batch_size = 50,verbose=1, sample_weight=None)

print ("Loss = " + str(preds[0])) 
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
predictimage = xtestsag[151]
predictimage=predictimage.reshape(1,256,256,1)
print(sagcnn.predict(predictimage))
print(ytestsagf[151])

In [ ]:
sagcnn04.save("sagpart1.h5")

In [ ]:
sagcnn.save("/kaggle/working/sagcnn_new.h5")

In [ ]:
from IPython.display import FileLink
FileLink(r'sagpart1.h5')

In [ ]:
sagcnn = keras.models.load_model("../input/ax-cnn/sagpart.h5")

# **AGGREGATION**

In [ ]:
axcnn = keras.models.load_model("../input/ax-cnn/axcnn_new.h5")

In [ ]:
corcnn = keras.models.load_model("../input/ax-cnn/corcnn_new.h5")

In [ ]:
sagcnn = keras.models.load_model("../input/ax-cnn/sagcnn_new.h5")

In [ ]:
sagcnn = keras.models.load_model("../input/ax-cnn/sagpart.h5")

In [ ]:
acl_labels = []
normal_labels = []
abnormal_labels = []
for i in range(1130):
    if labels_train[i,0] == 1 and labels_train[i,1] == 1:
        acl_labels.append(i)
    elif labels_train[i,0] == 0 and labels_train[i,1] == 0:
        normal_labels.append(i)
    else:
        abnormal_labels.append(i)

In [ ]:
acl_labels = np.array(acl_labels)
normal_labels = np.array(normal_labels)
abnormal_labels = np.array(abnormal_labels)

In [ ]:
len(abnormal_labels)

In [ ]:
aggregate_labels = np.concatenate((acl_labels[100:],normal_labels[150:],abnormal_labels[650]))

In [ ]:
normal_labels_cropped = normal_labels[:100]
abnormal_labels_cropped = abnormal_labels[:250]

In [ ]:
axial_data =[]
for i in acl_labels:
    temp = X_Train_ax[i,:,:,:]
    axial_data.append(temp)

In [ ]:
axial_data = np.array(axial_data)

In [ ]:
axial_data.shape

In [ ]:
axial_data.reshape(558,16,256,256)

In [ ]:
for i in normal_labels_cropped:
    print(i)
    axial_data.append(X_Train_ax[i,:,:,:])

In [ ]:
for i in abnormal_labels_cropped:
    print(i)
    axial_data.append(X_Train_ax[i,:,:,:])

In [ ]:
temp = axial_data[250,0,:,:]
temp = temp.reshape(1,256,256,1)
prediction = axcnn.predict(temp)
print(prediction)

In [ ]:
axial_a_agg = []
for i in range(558):
    predict_sum = 0
    for j in range(16):
        temp = axial_data[i,j,:,:]
        temp = temp.reshape(1,256,256,1)
        prediction = axcnn.predict(temp)
        predict_sum+=prediction
    predict_sum=predict_sum/16
    axial_a_agg.append(predict_sum)

In [ ]:
axial_b_agg = np.array(axial_b_agg)

In [ ]:
axial_a_agg = np.array(axial_a_agg)

In [ ]:
axial_a_agg[500]

In [ ]:
axial_a_agg.shape

In [ ]:
import pandas as pd

In [ ]:
areshape = axial_a_agg.reshape(558,3)

In [ ]:
areshape

In [ ]:
axial_a_agg_df = pd.DataFrame(data = areshape,columns = ["ax_normal","ax_abnormal","ax_acl"])

In [ ]:
print(axial_a_agg_df)

In [ ]:
axial_a_agg_df.to_csv('axial_aggregate_new.csv')

In [ ]:
from IPython.display import FileLink
FileLink(r'axial_aggregate.csv')

In [ ]:
cor_data =[]
for i in acl_labels:
    print(i)
    cor_data.append(X_Train_cor[i,:,:,:])
for i in normal_labels_cropped:
    print(i)
    cor_data.append(X_Train_cor[i,:,:,:]) 
for i in abnormal_labels_cropped:
    print(i)
    cor_data.append(X_Train_cor[i,:,:,:])    

In [ ]:
cor_data = np.array(cor_data)

In [ ]:
cor_data.shape

In [ ]:
cor_a_agg = []
for i in range(558):
    predict_sum = 0
    for j in range(16):
        temp = cor_data[i,j,:,:]
        temp = temp.reshape(1,256,256,1)
        prediction = corcnn.predict(temp)
        predict_sum+=prediction
    predict_sum=predict_sum/16
    cor_a_agg.append(predict_sum)

In [ ]:
cor_a_agg = np.array(cor_a_agg)
creshape = cor_a_agg.reshape(558,3)

In [ ]:
creshape[259]

In [ ]:
cor_a_agg_df = pd.DataFrame(data = creshape,columns = ["cor_normal","cor_abnormal","cor_acl"])

In [ ]:
print(cor_a_agg_df)

In [ ]:
cor_a_agg_df.to_csv('coronal_aggregate_new.csv')

In [ ]:
from IPython.display import FileLink
FileLink(r'coronal_aggregate.csv')

In [ ]:
sag_data =[]
for i in acl_labels:
    print(i)
    sag_data.append(X_Train_sag[i,:,:,:])
for i in normal_labels_cropped:
    print(i)
    sag_data.append(X_Train_sag[i,:,:,:]) 
for i in abnormal_labels_cropped:
    print(i)
    sag_data.append(X_Train_sag[i,:,:,:])    

In [ ]:
sag_data = np.array(sag_data)
sag_data.shape

In [ ]:
sag_a_agg = []
for i in range(558):
    predict_sum = 0
    for j in range(16):
        temp = sag_data[i,j,:,:]
        temp = temp.reshape(1,256,256,1)
        prediction = sagcnn.predict(temp)
        predict_sum+=prediction
    predict_sum=predict_sum/16
    sag_a_agg.append(predict_sum)

In [ ]:
sag_a_agg = np.array(sag_a_agg)
sreshape = sag_a_agg.reshape(558,3)

In [ ]:
sag_a_agg_df = pd.DataFrame(data = sreshape,columns = ["sag_normal","sag_abnormal","sag_acl"])

In [ ]:
print(sag_a_agg_df)

In [ ]:
sag_a_agg_df.to_csv('sagittal_aggregate_new.csv')

In [ ]:
FileLink(r'sagittal_aggregate.csv')

In [ ]:
s_label = []
for i in range(208):
    s_label.append(2)
for i in range(100):
    s_label.append(0)
for i in range(250):
    s_label.append(1)    

In [ ]:
s_label = np.array(s_label)
s_label.shape

In [ ]:
s_label.reshape(558,1)

In [ ]:
s_label_df = pd.DataFrame(data = s_label,columns = ["label"])

In [ ]:
s_label_df

In [ ]:
final_dataset = pd.concat([axial_a_agg_df,cor_a_agg_df,sag_a_agg_df,s_label_df],axis = 1)

In [ ]:
final_dataset.to_csv('softmax_dataset_new.csv')

In [ ]:
FileLink(r'softmax_dataset.csv')

In [ ]:
final_dataset

**Test**

In [ ]:
acl_test_labels = []
normal_test_labels = []
abnormal_test_labels = []
for i in range(119):
    if labels_train[i,0] == 1 and labels_train[i,1] == 1:
        acl_test_labels.append(i)
    elif labels_train[i,0] == 0 and labels_train[i,1] == 0:
        normal_test_labels.append(i)
    else:
        abnormal_test_labels.append(i)

In [ ]:
acl_test_labels = np.array(acl_test_labels)
normal_test_labels = np.array(normal_test_labels)
abnormal_test_labels = np.array(abnormal_test_labels)

In [ ]:
print(acl_test_labels.shape,normal_test_labels.shape,abnormal_test_labels.shape)

In [ ]:
aggregate_test_labels = np.concatenate((acl_test_labels,normal_test_labels,abnormal_test_labels))

In [ ]:
axial_test_data =[]
for i in acl_test_labels:
    axial_test_data.append(X_Test_ax[i,:,:,:])

In [ ]:
for i in normal_test_labels:
    axial_test_data.append(X_Train_ax[i,:,:,:])

In [ ]:
for i in abnormal_test_labels:
    print(i)
    axial_test_data.append(X_Train_ax[i,:,:,:])

In [ ]:
axial_test_data = np.array(axial_test_data)

In [ ]:
axial_test_data.shape

In [ ]:
axial_test_agg = []
for i in range(119):
    predict_sum = 0
    for j in range(16):
        temp = axial_test_data[i,j,:,:]
        temp = temp.reshape(1,256,256,1)
        prediction = axcnn.predict(temp)
        predict_sum+=prediction
    predict_sum=predict_sum/16
    axial_test_agg.append(predict_sum)

In [ ]:
axial_test_agg = np.array(axial_test_agg)
axial_test_agg = axial_test_agg.reshape(119,3)

In [ ]:
axial_test_agg_df = pd.DataFrame(data = axial_test_agg,columns = ["ax_normal","ax_abnormal","ax_acl"])

In [ ]:
axial_test_agg_df

In [ ]:
cor_test_data =[]
for i in acl_test_labels:
    cor_test_data.append(X_Test_cor[i,:,:,:])

In [ ]:
for i in normal_test_labels:
    cor_test_data.append(X_Train_cor[i,:,:,:])

In [ ]:
for i in abnormal_test_labels:
    cor_test_data.append(X_Train_cor[i,:,:,:])

In [ ]:
cor_test_data = np.array(cor_test_data)

In [ ]:
cor_test_agg = []
for i in range(119):
    predict_sum = 0
    for j in range(16):
        temp = cor_test_data[i,j,:,:]
        temp = temp.reshape(1,256,256,1)
        prediction = corcnn.predict(temp)
        predict_sum+=prediction
    predict_sum=predict_sum/16
    cor_test_agg.append(predict_sum)

In [ ]:
cor_test_agg = np.array(cor_test_agg)
cor_test_agg = cor_test_agg.reshape(119,3)

In [ ]:
cor_test_agg_df = pd.DataFrame(data = cor_test_agg,columns = ["cor_normal","cor_abnormal","cor_acl"])

In [ ]:
cor_test_agg_df


In [ ]:
sag_test_data =[]
for i in acl_test_labels:
    sag_test_data.append(X_Test_sag[i,:,:,:])

In [ ]:
for i in normal_test_labels:
    sag_test_data.append(X_Train_sag[i,:,:,:])

In [ ]:
for i in abnormal_test_labels:
    sag_test_data.append(X_Train_sag[i,:,:,:])

In [ ]:
sag_test_data = np.array(sag_test_data)

In [ ]:
sag_test_agg = []
for i in range(119):
    predict_sum = 0
    for j in range(16):
        temp = sag_test_data[i,j,:,:]
        temp = temp.reshape(1,256,256,1)
        prediction = sagcnn.predict(temp)
        predict_sum+=prediction
    predict_sum=predict_sum/16
    sag_test_agg.append(predict_sum)

In [ ]:
sag_test_agg = np.array(sag_test_agg)
sag_test_agg = sag_test_agg.reshape(119,3)

In [ ]:
sag_test_agg_df = pd.DataFrame(data = sag_test_agg,columns = ["sag_normal","sag_abnormal","sag_acl"])

In [ ]:
sag_test_agg_df

In [ ]:
final_test_dataset = pd.concat([axial_test_agg_df,cor_test_agg_df,sag_test_agg_df,s_test_label_df],axis = 1)

In [ ]:
s_test_label = []
for i in range(16):
    s_test_label.append(2)
for i in range(23):
    s_test_label.append(0)
for i in range(80):
    s_test_label.append(1)    

In [ ]:
s_test_label = np.array(s_test_label)
s_test_label.reshape(119,1)

In [ ]:
s_test_label_df = pd.DataFrame(data = s_test_label,columns = ["label"])

In [ ]:
final_test_dataset.to_csv('softmax_test_new.csv')

In [ ]:
FileLink(r'softmax_test.csv')